In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# Identify and specify the GPU as the device
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Install packages**

In [ ]:
 #!pip install pytorch_pretrained_bert
 #!pip install transformers==3.5.1
 !pip install nltk
import os
import logging
import argparse
import random
import json
import sklearn.metrics
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

import nltk

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


from transformers import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel, BertPreTrainedModel, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam


logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

# **Helper Functions**

In [ ]:
def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

# specifically for asc task

def create_examples(lines, set_type):
    examples = []
    for (i, ids) in enumerate(lines["data"]):
        guid = "%s-%s" % (set_type, ids )
        text_a = lines["data"][ids]['term']
        text_b = lines["data"][ids]['sentence']
        label = lines["data"][ids]['polarity']
        examples.append(
            InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label) )
    return examples   

def read_json(input_file):
    """Reads a json file for tasks in sentiment analysis."""
    with open(input_file) as f:
        return json.load(f)
        
def get_train_examples(data_dir):
    """See base class."""
    return create_examples(
        read_json(data_dir), "train")

def get_dev_examples(data_dir):
    """See base class."""
    return create_examples(
        read_json(data_dir), "dev")    

def get_test_examples(data_dir):
    """See base class."""
    return create_examples(
        read_json(data_dir), "test")    
    
class BertForSequenceLabeling(BertPreTrainedModel):
    def __init__(self, config, num_labels=3):
        super(BertForSequenceLabeling, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        sequence_output, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-1)
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, mode):
    """Loads a data file into a list of `InputBatch`s.""" #check later if we can merge this function with the SQuAD preprocessing 
    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    features = []
    for (ex_index, example) in enumerate(examples):
        if mode!="ae":
            tokens_a = tokenizer.tokenize(example.text_a)
        else: #only do subword tokenization.
            tokens_a, labels_a, example.idx_map= tokenizer.subword_tokenize([token.lower() for token in example.text_a], example.label )

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)]

        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        if mode!="ae":
            label_id = label_map[example.label]
        else:
            label_id = [-1] * len(input_ids) #-1 is the index to ignore
            #truncate the label length if it exceeds the limit.
            lb=[label_map[label] for label in labels_a]
            if len(lb) > max_seq_length - 2:
                lb = lb[0:(max_seq_length - 2)]
            label_id[1:len(lb)+1] = lb

        features.append(
                InputFeatures(
                        input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id))
    return features

class ABSATokenizer(BertTokenizer):     
    def subword_tokenize(self, tokens, labels): # for AE
        split_tokens, split_labels= [], []
        idx_map=[]
        for ix, token in enumerate(tokens):
            sub_tokens=self.wordpiece_tokenizer.tokenize(token)
            for jx, sub_token in enumerate(sub_tokens):
                split_tokens.append(sub_token)
                if labels[ix]=="B" and jx>0:
                    split_labels.append("I")
                else:
                    split_labels.append(labels[ix])
                idx_map.append(ix)
        return split_tokens, split_labels, idx_map  

### Text preprocessing         

LABEL_SET = {    
        "BIO": {
            "label_map": {
                "O": "O", 
                "B-positive": "B", 
                "B-negative": "B", 
                "B-neutral": "B",
                "B-conflict": "B",
                "I-positive": "I",
                "I-negative": "I",
                "I-neutral": "I",
                "I-conflict": "I",
            },
            "label_list": ["O", "B", "I"]
        },
        "AO": {
            "label_map": {
                "O": "O",
                "B-positive": "A", 
                "B-negative": "A", 
                "B-neutral": "A",
                "B-conflict": "A",
                "I-positive": "A",
                "I-negative": "A",
                "I-neutral": "A",
                "I-conflict": "A",
            },
            "label_list": ["O", "A"]
        },
        "PNNO": {
            "label_map": {
                "O": "O",
                "B-positive": "positive", 
                "B-negative": "negative", 
                "B-neutral": "neutral",
                "B-conflict": "neutral",
                "I-positive": "positive",
                "I-negative": "negative",
                "I-neutral": "neutral",
                "I-conflict": "neutral",
            },
            "label_list": ["O", "positive", "negative", "neutral"]
        }
    }


def _tokenize_text(raw_text, from_index, to_index, label_map, sentiment):
    # warning this code this borrowed from the project : https://github.com/howardhsu/DE-CNN
    # the code needs cleaner re-writing.
    text = []
    for ix, c in enumerate(raw_text):
        # assuming that the tokenizer always yields fine-grained tokens for aspects 
        # so tokenizer won't affect the performance of AE.
        if (c=='/' or c=='*' or c=='-' or c=='=') and len(text)>0 and text[-1]!=' ':
            text.append(' ')
        if ix==int(from_index ) and len(text)>0 and text[-1]!=' ':
            text.append(' ')
        elif ix==int(to_index ) and len(text)>0 and text[-1]!=' ' and c!=' ': 
            text.append(' ')
        text.append(c)
        if (c=='/' or c=='*' or c=='-' or c=='=') and text[-1]!=' ':
            text.append(' ')

    text="".join(text)
    tokens=nltk.word_tokenize(text, language='german')
    lb = [label_map["O"]]*len(tokens)
    
    token_idx, pt, tag_on=0, 0, False
    for ix, c in enumerate(raw_text):
        if pt>=len(tokens[token_idx] ):
            pt=0
            token_idx+=1
            if token_idx >= len(tokens):
                break
        if ix==from_index: #from
            assert pt == 0 and c != ' '
            lb[token_idx] = label_map["B-" + sentiment]
            tag_on = True
        elif ix==to_index: #to
            assert pt == 0
            tag_on = False
        elif tag_on and pt==0 and c!=' ':
            lb[token_idx] = label_map["I-" + sentiment]
        if c==' ' or ord(c)==160: # skip spaces.
            pass
        elif tokens[token_idx][pt:pt+2]=='``' or tokens[token_idx][pt:pt+2]=="''":
            pt+=2
        else:
            pt+=1
    return tokens, lb

def ae_task(data):
    corpus = []
    for index, tweet in data.iterrows():
        tokens, lb = _tokenize_text(raw_text = tweet['full_text'], 
                                from_index = tweet['from'], 
                                to_index = tweet['to'],
                                label_map = LABEL_SET["BIO"]["label_map"],
                                sentiment = tweet['label'])
        corpus.append({"id": tweet['doc_id'], 
                       "tokens": tokens, 
                       "labels": lb}) 
        
    return corpus

def asc_task(data):
    corpus = []
    for index, tweet in data.iterrows():
        corpus.append({"id": tweet['doc_id'], 
                       "sentence": tweet['full_text'], 
                       "term": tweet['topic'], 
                       "polarity": tweet['label']})     
        
    return corpus

def write_json(filename, corpus, meta=['O', 'B', 'I'], target_dir = 'D:/UNI/01_Master/3. Semester/Consulting/00_BERT/'):
    path = os.path.join(target_dir, filename)
    with open(path, "w") as fw:
        json.dump({"data": {rec["id"]: rec for rec in corpus}, "meta": meta}, fw)    

### Evaluation part

def display_confusion_matrix(true_labels, predicted_labels, classes=[1,0]):
    
    cm = confusion_matrix(y_true=true_labels, 
                                  y_pred=predicted_labels, 
                                  labels=classes)
    cm_frame = pd.DataFrame(data=cm, 
                            columns=pd.MultiIndex(levels=[['Predicted:'], classes], 
                                                  codes=[[0,0],[0,1]]), 
                            index=pd.MultiIndex(levels=[['Actual:'], classes], 
                                                codes=[[0,0],[0,1]])) 
    return cm_frame   

# **Set Arguments**

In [ ]:
train_batch_size = 32            
num_train_epochs = 4      
max_seq_length = 100
learning_rate = 3e-5
warmup_proportion = 0.1
do_valid = True
pretrained_model = "bert-base-german-cased"

# **Training Setup**

In [ ]:
label_list = ["positive", "negative", "neutral"] 

tokenizer = ABSATokenizer.from_pretrained(pretrained_model)
train_examples = get_train_examples(data_dir = "train.json")
num_train_steps = int(len(train_examples) / train_batch_size) * num_train_epochs

train_features = convert_examples_to_features(examples = train_examples, 
                                              label_list = label_list, 
                                              max_seq_length = max_seq_length, 
                                              tokenizer = tokenizer, 
                                              mode = "asc")
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_steps)

all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

train_data = TensorDataset(all_input_ids, all_segment_ids, all_input_mask, all_label_ids)

train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

02/09/2021 23:34:25 - INFO - filelock -   Lock 140192104248096 acquired on /root/.cache/torch/transformers/da299cdd121a3d71e1626f2908dda0d02658f42e925a3d6abd8273ec08cf41a6.31ccc255fc2bad3578089a3997f16b286498ba78c0adc43b5bb2a3f9a0d2c85c.lock


02/09/2021 23:34:26 - INFO - filelock -   Lock 140192104248096 released on /root/.cache/torch/transformers/da299cdd121a3d71e1626f2908dda0d02658f42e925a3d6abd8273ec08cf41a6.31ccc255fc2bad3578089a3997f16b286498ba78c0adc43b5bb2a3f9a0d2c85c.lock
02/09/2021 23:34:26 - INFO - __main__ -   ***** Running training *****
02/09/2021 23:34:26 - INFO - __main__ -     Num examples = 2
02/09/2021 23:34:26 - INFO - __main__ -     Batch size = 32
02/09/2021 23:34:26 - INFO - __main__ -     Num steps = 0


# **Validation Setup**

In [ ]:
valid_examples = get_dev_examples(data_dir = "dev.json")
valid_features= convert_examples_to_features(
    valid_examples, label_list,  max_seq_length, tokenizer, "asc")
valid_all_input_ids = torch.tensor([f.input_ids for f in valid_features], dtype=torch.long)
valid_all_segment_ids = torch.tensor([f.segment_ids for f in valid_features], dtype=torch.long)
valid_all_input_mask = torch.tensor([f.input_mask for f in valid_features], dtype=torch.long)
valid_all_label_ids = torch.tensor([f.label_id for f in valid_features], dtype=torch.long)
valid_data = TensorDataset(valid_all_input_ids, valid_all_segment_ids, valid_all_input_mask, valid_all_label_ids)

logger.info("***** Running validations *****")
logger.info("  Num orig examples = %d", len(valid_examples))
logger.info("  Num split examples = %d", len(valid_features))
logger.info("  Batch size = %d",  train_batch_size)

valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size= train_batch_size)    

best_valid_loss=float('inf')
valid_losses=[]

02/09/2021 23:34:50 - INFO - __main__ -   ***** Running validations *****
02/09/2021 23:34:50 - INFO - __main__ -     Num orig examples = 3
02/09/2021 23:34:50 - INFO - __main__ -     Num split examples = 3
02/09/2021 23:34:50 - INFO - __main__ -     Batch size = 32


# **Optimization & Actual Training**

In [ ]:
    model = BertForSequenceClassification.from_pretrained("https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased.tar.gz", num_labels = len(label_list))
    model.cuda()
    # Prepare optimizer
    param_optimizer = [(k, v) for k, v in model.named_parameters() if v.requires_grad==True]
    param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    t_total = num_train_steps
    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=learning_rate,
                         warmup= warmup_proportion,
                         t_total=t_total)

02/09/2021 23:36:09 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased.tar.gz from cache at /root/.pytorch_pretrained_bert/4983b4079d9a070d00f914e5e669c1b1376f201367e3507366bc8a9de9d62fd0.d994c35add5d5ceb797a0bce40d17a4abb02c9c37b804e92c448d31d8de34734
02/09/2021 23:36:09 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/4983b4079d9a070d00f914e5e669c1b1376f201367e3507366bc8a9de9d62fd0.d994c35add5d5ceb797a0bce40d17a4abb02c9c37b804e92c448d31d8de34734 to temp dir /tmp/tmpx7ef8nns
02/09/2021 23:36:12 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_

In [ ]:
global_step = 0
model.train()
for _ in range(num_train_epochs):
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.cuda() for t in batch)
        input_ids, segment_ids, input_mask, label_ids = batch
        loss = model(input_ids, segment_ids, input_mask, label_ids)
        loss.backward()

        lr_this_step = learning_rate * warmup_linear(global_step/t_total, warmup_proportion)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr_this_step
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1
        #>>>> perform validation at the end of each epoch .
    if do_valid:
        model.eval()
        with torch.no_grad():
            losses=[]
            valid_size=0
            for step, batch in enumerate(valid_dataloader):
                batch = tuple(t.cuda() for t in batch) # multi-gpu does scattering it-self
                input_ids, segment_ids, input_mask, label_ids = batch
                loss = model(input_ids, segment_ids, input_mask, label_ids)
                losses.append(loss.data.item()*input_ids.size(0) )
                valid_size+=input_ids.size(0)
            valid_loss=sum(losses)/valid_size
            logger.info("validation loss: %f", valid_loss)
            valid_losses.append(valid_loss)
        if valid_loss<best_valid_loss:
            torch.save(model, "model.pt")
            best_valid_loss=valid_loss
        model.train()
if do_valid:
    with open("valid.json", "w") as fw:
        json.dump({"valid_losses": valid_losses}, fw)
else:
    torch.save(model, "model.pt")

/usr/local/lib/python3.6/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
02/09/2021 23:36:23 - INFO - __main__ -   validation loss: 1.117757
02/09/2021 23:36:25 - INFO - __main__ -   validation loss: 0.960526
02/09/2021 23:36:26 - INFO - __main__ -   validation loss: 0.867670
02/09/2021 23:36:28 - INFO - __main__ -   validation loss: 0.827143


# **Final Evaluation**

This is the End to End Analysis, the combined AE and ASC tasks will be evaluated in this section: 


1.   Insert original Tweets without topics and labels: tweets
2.   Insert predicted and post-processed aspects: y_pred_aspects
3.   Insert tweets in tokenized form during AE task: pred_json["raw_X"]
4.   Result: Aspect Extraction. Fill in the topic-column

In [ ]:
with open("predictions_ae.json") as f:
    pred_json=json.load(f)   
y_pred_aspects=[]
for ix, logit in enumerate(pred_json["logits"]):
    pred=[0]*len(pred_json["raw_X"][ix])
    for jx, idx in enumerate(pred_json["idx_map"][ix]):
        lb=np.argmax(logit[jx])
        if lb==1: #B
            pred[idx]=1
        elif lb==2: #I
            if pred[idx]==0: #only when O->I (I->I and B->I ignored)
                pred[idx]=2
    y_pred_aspects.append(pred)

In [ ]:
import pandas as pd

tweets = pd.read_csv('bert_toy.csv') 
tweets = tweets[0:2]
tweets['topic'] = ""
label = y_pred_aspects
corpus = pred_json["raw_X"]
pred_y=[] #zx, sent 
for zx, tweet in tweets.iterrows():
    tokens = corpus[zx]
    lb = label[zx]
    # opin = tweet['topic']
    token_idx, pt, tag_on=0, 0, False
    start, end=-1, -1
    for ix, c in enumerate(tweet['full_text']):
        if token_idx<len(tokens) and pt>=len(tokens[token_idx] ):
            pt=0
            token_idx+=1

        if token_idx<len(tokens) and lb[token_idx]==1 and pt==0 and c!=' ':
            if tag_on:
                end=ix
                tag_on=False
                #opin=ET.Element("aspectTerm")
                tweet['topic']=tweet['full_text'][start:end]
                tweet["from"]=str(start)
                tweet['to']=str(end)
                #opins.append(opin)
            start=ix
            tag_on=True
        elif token_idx<len(tokens) and lb[token_idx]==2 and pt==0 and c!=' ' and not tag_on:
            start=ix
            tag_on=True
        elif token_idx<len(tokens) and (lb[token_idx]==0 or lb[token_idx]==1) and tag_on and pt==0:
            end=ix
            tag_on=False 
            #opin=ET.Element("aspectTerm")
            tweet['topic']=tweet['full_text'][start:end]
            tweet["from"]=str(start)
            tweet['to']=str(end)
            #opins.append(opin)
        elif token_idx>=len(tokens) and tag_on:
            end=ix
            tag_on=False 
            #opin=ET.Element("aspectTerm")
            tweet['topic']=tweet['full_text'][start:end]
            tweet["from"]=str(start)
            tweet['to']=str(end)
            #opins.append(opin)
        if c==' ' or ord(c)==160:
            pass
        elif tokens[token_idx][pt:pt+2]=='``' or tokens[token_idx][pt:pt+2]=="''":
            pt+=2
        else:
            pt+=1
    if tag_on:
        tag_on=False
        end=len(tweet['full_text'])
        #opin=ET.Element("aspectTerm")
        tweet['topic']=tweet['full_text'][start:end]
        tweet["from"]=str(start)
        tweet['to']=str(end)
        #opins.append(opin)
    tweets.at[zx,'topic'] = tweet['topic'] 

Prepare test examples based on topics, gained due to AE Task    

In [ ]:
test_corpus = asc_task(data = tweets) 
write_json("test.json", corpus = test_corpus, target_dir = "")

eval_batch_size = 8
eval_examples = get_test_examples(data_dir = "test.json")
eval_features = convert_examples_to_features(eval_examples, label_list, max_seq_length, tokenizer, "asc")

In [ ]:


logger.info("***** Running evaluation *****")
logger.info("  Num examples = %d", len(eval_examples))
logger.info("  Batch size = %d", eval_batch_size)
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_segment_ids, all_input_mask, all_label_ids)
# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size= eval_batch_size)

model = torch.load("model.pt" )
model.cuda()
model.eval()

full_logits=[]
full_label_ids=[]
for step, batch in enumerate(eval_dataloader):
    batch = tuple(t.cuda() for t in batch)
    input_ids, segment_ids, input_mask, label_ids = batch
    
    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask)

    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.cpu().numpy()

    full_logits.extend(logits.tolist() )
    full_label_ids.extend(label_ids.tolist() )

with open("predictions_asc.json", "w") as fw:
    json.dump({"logits": full_logits, "label_ids": full_label_ids}, fw)

02/09/2021 23:37:22 - INFO - __main__ -   ***** Running evaluation *****
02/09/2021 23:37:22 - INFO - __main__ -     Num examples = 2
02/09/2021 23:37:22 - INFO - __main__ -     Batch size = 8


# **Evaluation Metrics**

In [ ]:
# Various evaluation metrics

with open( "predictions_asc.json" ) as f:
    results=json.load(f)
y_true=results['label_ids']
y_pred=[np.argmax(logit) for logit in results['logits'] ]
p_macro, r_macro, f_macro, _=sklearn.metrics.precision_recall_fscore_support(y_true, y_pred, average='macro')
f_macro = 2*p_macro*r_macro/(p_macro+r_macro)
acc = 100*sklearn.metrics.accuracy_score(y_true, y_pred)
f_mac = 100*f_macro 

# Confusion Matrix
display_confusion_matrix(true_labels = y_true, predicted_labels = y_pred)


Predicted:   
                   1  0
Actual: 1          1  0
        0          0  1